# [Object-Oriented Programming in Python](https://www.datacamp.com/completed/statement-of-accomplishment/course/d521f6dde77df741bef402e1d2fc9c95e6f542f1)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adamelliotfields/datacamp/blob/main/notebooks/python/oop_in_python/notebook.ipynb)

In [1]:
from abc import ABC, abstractmethod


class GrandParent1(ABC):
    """An Abstract Base Class (ABC) that cannot be instantiated."""

    @abstractmethod
    def instance_method(self):
        """Any child classes must implement this method."""
        pass


class GrandParent2:
    def grandparent_method(self):
        """This method is inherited by all children unless they override it.
        It can be called via `super()`.
        """
        print("Method in GrandParent2")


class Parent1(GrandParent1):
    """A parent class that inherits from an ABC.

    Class Attributes:
        LAST_NAME (str): The last name of the parent. Class attributes are shared by all instances
                         of the class.

    Attributes:
        first_name (str): The first name of the parent.
    """

    LAST_NAME = "Doe"

    def __init__(self, first_name):
        """Constructor for Parent1.

        Args:
            first_name (str): A string to set as the first name of the parent.
        """
        self.first_name = first_name

    @staticmethod
    def static_method():
        """Static methods are not class-aware (ie, no "cls" or "self").
        They are inherited and can be overridden.
        """
        print("Static method in Parent1")

    def instance_method(self):
        """An instance method that overrides the one in GrandParent1.
        Required to implement because we inherit directly from GrandParent1 and it is abstract.
        """
        print(f"Instance method in {self.__class__.__name__}")


class Parent2(GrandParent2):
    """A parent class that is not abstract.

    Class Attributes:
        LAST_NAME (str): The last name of the parent.

    Properties:
        age (int): The age of the parent.
    """

    LAST_NAME = "Smith"

    def __init__(self, age):
        """Constructor for Parent2.

        Args:
            age (int): An integer to set as the age of the parent.
        """
        self.age = age

    @classmethod
    def class_method(cls):
        """Class methods are class-aware, unlike their static counterparts.
        They are inherited and can be overridden.
        """
        print(f"Class method in {cls.__name__}")

    @property
    def age(self):
        """The `@property` decorator allows us to use `age` as an attribute instead of a method.
        If we don't define a setter, then the property is read-only.

        Returns:
            int: The age of the parent.
        """
        return self._age

    @age.setter
    def age(self, value):
        """Setter for `age` property. Note that "getters" and "deleters" can also be defined.

        Args:
            value (int): The age of the parent.

        Raises:
            ValueError: If the value passed in is less than 0.
        """
        if value < 0:
            raise ValueError("Age cannot be negative")

        self._age = value  # set the "private" attribute `_age`


class Child(Parent1, Parent2):
    """A child class that inherits from multiple parents.

    Attributes:
        first_name (str): The first name of the child.
        age (int): The age of the child.
    """

    def __init__(self, first_name, age):
        """Constructor for Child.

        We explicity call each parent's constructor instead of using `super()` because we need to
        inherit from both.

        When you call `super()`, Python's Module Resolution Order (MRO) determines which parent's
        method is called.

        Args:
            first_name (str): A string to set as the name of the child.
            age (int): An integer to set as the age of the child.
        """
        Parent1.__init__(self, first_name)
        Parent2.__init__(self, age)

    # NB: There are other "special" methods that you can override besides `__init__()`.
    # https://docs.python.org/3/reference/datamodel.html#special-method-names
    def __str__(self):
        """The `__str__()` method is called when you use `str(obj)` or `print(obj)`."""
        return f"Name: {self.first_name} {self.LAST_NAME}, Age: {self.age}"

    def __repr__(self):
        """The `__repr__()` method is called when you use `repr(obj)`."""
        return (
            f"{self.__class__.__name__}(name='{self.first_name} {self.LAST_NAME}',age={self.age})"
        )

    def __eq__(self, other):
        """The `__eq__()` method is called when you use `obj1 == obj2`."""
        return self.first_name == other.first_name and self.age == other.age

    def __ne__(self, other):
        """The `__ne__()` method is called when you use `obj1 != obj2`."""
        return not self.__eq__(other)

    def instance_method(self):
        """An instance method that overrides the one in Parent1."""
        print(f"Instance method in {self.__class__.__name__}")
        super().static_method()  # "Static method in Parent1"
        super().class_method()  # "Class method in Child"
        super().grandparent_method()  # "Method in GrandParent2"


class GrandChild(Child):
    """A grandchild class that inherits from Child.

    Class Attributes:
        LAST_NAME (str): The last name of the grandchild.

    Attributes:
        first_name (str): The first name of the grandchild.
        age (int): The age of the grandchild.
    """

    LAST_NAME = "Lovelace"

    def __init__(self, first_name, age):
        """Constructor for GrandChild.

        Can use `super()` because it only inherits from a single parent (Child).

        Args:
            first_name (str): A string to set as the first name of the grandchild.
            age (int): An integer to set as the age of the grandchild.
        """
        super().__init__(first_name, age)

    def __pseudo_private_method(self):
        """Psuedo-private methods are prefixed with a double-underscore.
        These methods are "mangled" and can only be accessed via `obj._ClassName__method()`.
        (applies to attributes as well)
        """
        print("Pseudo-private method in GrandChild")

    def _internal_method(self):
        """Internal methods are prefixed with a single underscore.
        They are also known as "protected" methods.
        Note that they are not truly "internal" as they can still be accessed from outside the class.
        """
        print("Internal method in GrandChild")
        self.__pseudo_private_method()  # can call pseudo-private method from within the class

In [2]:
# Create an instance of Child and GrandChild
child = Child("John", 30)
grand_child = GrandChild("Ada", 42)

In [3]:
# Check age setter
try:
    time_traveler = Child("Marty", -1)
except ValueError as e:
    print(e)  # "Age cannot be negative"

try:
    child.age = -1
except ValueError as e:
    print(e)  # "Age cannot be negative"

print(child.age)  # 30 (unchanged)

Age cannot be negative
Age cannot be negative
30


In [4]:
# Call static methods
Child.static_method()  # calls `Parent1.static_method()` because we didn't override it
Parent1.static_method()

Static method in Parent1
Static method in Parent1


In [5]:
# Call class methods
Child.class_method()  # prints "Child" not "Parent2" because it is class-aware
Parent2.class_method()

Class method in Child
Class method in Parent2


In [6]:
# Call instance methods
child.instance_method()

Instance method in Child
Static method in Parent1
Class method in Child
Method in GrandParent2


In [7]:
# Call "private" methods (NB: you would never do this as it is a violation of encapsulation)
grand_child._internal_method()
grand_child._GrandChild__pseudo_private_method()

Internal method in GrandChild
Pseudo-private method in GrandChild
Pseudo-private method in GrandChild


In [8]:
# Print string representations
print(child)  # "Name: John Doe, Age: 30"
print(grand_child)  # "Name: Ada Lovelace, Age: 42"
print(repr(child))  # "Child(name='John Doe',age=30)"
print(repr(grand_child))  # "GrandChild(name='Ada Lovelace',age=42)"

Name: John Doe, Age: 30
Name: Ada Lovelace, Age: 42
Child(name='John Doe',age=30)
GrandChild(name='Ada Lovelace',age=42)


In [9]:
# Check the Module Resolution Order (MRO) of Child.
# Calling `super().class_method()` in Child will call the `class_method()` of Parent1.
Child.mro()  # list

[__main__.Child,
 __main__.Parent1,
 __main__.GrandParent1,
 abc.ABC,
 __main__.Parent2,
 __main__.GrandParent2,
 object]

In [10]:
Child.__mro__  # tuple

(__main__.Child,
 __main__.Parent1,
 __main__.GrandParent1,
 abc.ABC,
 __main__.Parent2,
 __main__.GrandParent2,
 object)

In [11]:
help(GrandChild)

Help on class GrandChild in module __main__:

class GrandChild(Child)
 |  GrandChild(first_name, age)
 |  
 |  A grandchild class that inherits from Child.
 |  
 |  Class Attributes:
 |      LAST_NAME (str): The last name of the grandchild.
 |  
 |  Attributes:
 |      first_name (str): The first name of the grandchild.
 |      age (int): The age of the grandchild.
 |  
 |  Method resolution order:
 |      GrandChild
 |      Child
 |      Parent1
 |      GrandParent1
 |      abc.ABC
 |      Parent2
 |      GrandParent2
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, first_name, age)
 |      Constructor for GrandChild.
 |      
 |      Can use `super()` because it only inherits from a single parent (Child).
 |      
 |      Args:
 |          first_name (str): A string to set as the first name of the grandchild.
 |          age (int): An integer to set as the age of the grandchild.
 |  
 |  -------------------------------------------------------------------